<a href="https://colab.research.google.com/github/EmmanuelADAM/IntelligenceArtificiellePython/blob/master/TPClassificationLivres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
---
# Classification de commentaires de livres (en français)

## Collection de données de critiques de films 
Le tp utilisera l'ensemble de données issues de 
[French Books Reviews](https://huggingface.co/datasets/Abirate/french_book_reviews) que vous devez télécharger. 

Ce fichier CSV contient presque 10000 commentaires sur des livres, leurs notes, et l'avis (positif, négatif ou neutre).

---
Vous vous baserez sur la solution à *[Détection de "sentiments"](https://github.com/EmmanuelADAM/IntelligenceArtificiellePython/blob/master/SolutionTPDetectionDeSentiments.ipynb)*.

Vous adapterez le code présent pour traiter le texte en français.
Simpement, les "stopwords" seront chargés ainsi : 

In [ ]:
stop_words = set(stopwords.words('french'))

---
A partir de la colonne 'reader_review' plusieurs classements sont possibles : 
- soit le classement de valeurs entre -1 et 1 (ce qui correspond à la colonne 'label')
- soit le classement par une note entre . et 5 (ce qui correspond à la colonne 'rating')

*N.B. la fonction d'activation sigmoid pren ses valeurs dans [0,1], l'activation en utilisant la tangente hyperboliquye 'tanh' prend ses valeus dans [-1, 1]*

---

## DEBUT DU TP
Voici quelques elements de code pour démarrer

### les bibliothèques

In [ ]:
##import pour les fichiers et le traitement de données : 
from os import listdir
import pandas as pd
from tqdm import tqdm

##import pour la creation de tokens
import nltk
from nltk.corpus import stopwords

##import pour la gestion de ponctuation
from string import punctuation

##import pour une collection spécifique : counter
from collections import Counter

##import pour les réseaux de neurones : 
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import preprocessing

##import classique pour les tableaux efficaces
import numpy as np

#quelques types, optionnels, juste pour un beau code
Series = pd.core.series.Series
DataFrame = pd.core.frame.DataFrame

### Nettoyage des données
On passe les textes dans un premier filtres pour ôter les mots qui sembles inutiles (stop words, ponctuation, etc)

In [ ]:
# le code suivant charge l'ensemble des mots non importants (en français) dans la variable stop_words
nltk.download("stopwords")
stop_words = set(stopwords.words('french'))
print(stop_words)

In [ ]:
# nettoie un document en supprimant les mots non importants et la ponctuation
# retourne le texte nettoyé sous forme de liste de mots
def clean_doc(doc:str)->list:
    """retourne la liste de mots clés inclus dans le texte doc 
    qui ne font pas parti des stop_words"""
    tokens = []
    if (type(doc)!=float):
        # split into tokens by white space  
        tokens = doc.split()
        # remove punctuation from each token
        table = str.maketrans(' ', ' ', punctuation)
        tokens = [w.translate(table) for w in tokens]
        # remove remaining tokens that are not alphabetic
        tokens = [word.lower() for word in tokens if word.isalpha()]
        # filter out stop words
        tokens = [w for w in tokens if not w in stop_words]
        # filter out short tokens
        tokens = [word for word in tokens if len(word) > 2]
    return tokens
 

### Tests de la récupération de mots clés d'un fichier

In [ ]:
rep = '/content/drive/My Drive/txt_sentoken'
# ou sous windows rep:str = "C:/VOTRE REPERTOIRE/"
# ou sous linux & mac os rep:str = "/VOTRE REPERTOIRE/"

# recuperer les donnees du fichier grace a pandas
filename:str = "french_books_reviews.json"
datas:DataFrame = pd.read_json(rep+filename)

# recuperer la colonne des reviews (cf. le nom des colonnes dans le json ou dans la page decrivant le dataset)
reviews:Series = datas['reader_review']

In [ ]:
# TEST DE LA FONCTION clean_doc sur le premier texte
tokens:list = clean_doc(reviews[0])
print('les 10 premiers mots cles de ', reviews[0])
print(tokens[:10])